In [3]:
from PIL import Image
from pathlib import Path
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

from clearml import Task

## ClearML agent

In [2]:
task = Task.init(project_name='VR Mental Health Clinic', 
    task_name='Audio IFS Plots - Pretrained ResNet')
config_dict = {'num_of_epochs': 10, 'batch_size': 8, 'drop_out': 0.25, 'lr': 5e-5,
               'save_dir': Path('./results/20210520-peudo-eyetracking-cnn')}
config_dict = task.connect(config_dict)
print(config_dict)

data_dir = Path('./data/audio')

ClearML Task: created new task id=0d5ed1cda5744de2b08e806f42d54bc3
2022-10-13 13:19:48,524 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.community.clear.ml/projects/f99841227bad4da5b63d3ca4147ddaf0/experiments/0d5ed1cda5744de2b08e806f42d54bc3/output/log
2022-10-13 13:19:49,063 - clearml.Task - WARNING - Skipping parameter: General/save_dir[WindowsPath], only builtin types are supported (NoneType, dict, list, tuple, str, int, float)
{'num_of_epochs': 10, 'batch_size': 8, 'drop_out': 0.25, 'lr': 5e-05, 'save_dir': WindowsPath('results/20210520-peudo-eyetracking-cnn')}
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


## Dataset class

In [ ]:
class AudioIFSDataSet(Dataset):
    def __init__(self,meta_df_path,data_dir):
        self.paths = []
        self.labels = []
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
        
        meta_df = pd.read_csv(meta_df_path)
        num_id = 8
        for _, row in meta_df.iterrows():
            f = row['file']
            l = row['label']
            self.paths.append(data_dir/(f+'.png'))
            self.labels.append(l)
            
            for i in range(num_id):
                self.paths.append(data_dir/(f+'_'+str(i+1)+'.png'))
                self.labels.append(l)
                
                for j in range(num_id):
                    self.paths.append(data_dir/(f+'_'+str(i+1)+str(j+1)+'.png'))
                    self.labels.append(l)
                    
    def __len__(self):
        return len(self.paths)

    def __getitem__(self,idx):
        ifs_img = Image.open(self.paths[idx])
        ifs_img = ifs_img.convert('RGB')
        ifs_img = self.transform(ifs_img)
        
        return ifs_img, self.labels[idx]

### Data preparation

In [23]:
data_dir = Path('./data/audio')

# data sets
train_ds = AudioIFSDataSet(data_dir/'meta_train.csv', data_dir) 
valid_ds = AudioIFSDataSet(data_dir/'meta_valid.csv', data_dir)
test_ds = AudioIFSDataSet(data_dir/'meta_test.csv', data_dir)

# data loaders
train_dl = DataLoader(train_ds, batch_size=config_dict.get('batch_size'))
valid_dl = DataLoader(valid_ds, batch_size=config_dict.get('batch_size'))
test_dl = DataLoader(test_ds, batch_size=config_dict.get('batch_size'))

In [ ]:
# pretrained ResNet
model = models.resnet34(pretrained=True) # try from the empty model? put this option in the configuratin dictionary?
model.fc = nn.Sequential(nn.Dropout(p=config_dict.get('drop_out')),
                         nn.Linear(512,1),
                         nn.Sigmoid())

# optimizer
optimizer = optim.Adam(model.parameters(), lr=config_dict.get('lr'))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=config_dict.get('num_epochs')//4, gamma=0.5)
criterion = nn.BCELoss()

# device (GPU) setting
